In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


df=pd.read_csv('../input/spotifydata5/data5.csv')
df.head(10)


In [ ]:
df.shape

In [ ]:
list(df.columns)

In [ ]:
#필요없는 컬럼 삭제
df=df.drop(columns=['key', 'explicit','mode','id'])
df.shape

In [ ]:
#컬럼 내 null값 확인 
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
print('                                  <인기도가 가장 높은 노래 순위>')
df[['name','artists','release_date','popularity','tempo']].sort_values(by='popularity',ascending=False)[:20]


해석: spotify에서 가장 인기가 많은 20곡 모두 90 이상의 빠른 템포를 보임. 
(*보통 템포가 90 이상이면 빠른 음악에 속함)

In [ ]:
plt.figure(figsize=(16,4))
sns.set(style="darkgrid")
x=df.groupby("name")["popularity"].max().sort_values(ascending=False).head(10)
ax=sns.barplot(x.index,x)
ax.set_title('The most popular songs on Spotify', fontsize=20)
ax.set_ylabel('Popularity', fontsize=15)
ax.set_xlabel('songs',fontsize=15)
plt.xticks(rotation=90)

스포티파이에서 가장 인기 많은 노래 순위 


In [ ]:
#SPOTIFY에서 가장 인기도가 높은 노래 순 

plt.figure(figsize=(16,4))
sns.set(style="darkgrid")
x=df.groupby("artists")["popularity"].max().sort_values(ascending=False).head(10)
ax=sns.barplot(x.index,x)
ax.set_title('The most popular artists on Spotify', fontsize=20)
ax.set_ylabel('Popularity', fontsize=15)
ax.set_xlabel('Artists', fontsize=15)
plt.xticks(rotation=90)

스포티파이에서 가장 인기 많은 가수 순위 

In [ ]:
print('가장 노래를 많이 발표한 가수')
song_counts=df.artists.value_counts()[:10]
song_counts 

In [ ]:
print('        <kpop 대표 아이돌그룹 블랙핑크, BTS, TWICE의 곡 상위 25 인기도 순위>')

xs=df[['name','artists','release_date','popularity']].sort_values(by='popularity',ascending=False)
kpops = xs['artists'].str.contains("BLACKPINK|BTS|TWICE")
kpop = xs[kpops]
kpop.head(25)


In [ ]:
df['duration_min']=df['duration_ms']/60000
grouped_music=df['duration_min'].groupby(df['year']).describe()
grouped_music

In [ ]:
avg_runtime_by_year=grouped_music['mean']
avg_runtime_min=grouped_music['mean']-grouped_music['std']
avg_runtime_max=grouped_music['mean']+grouped_music['std']

In [ ]:
fig, x=plt.subplots(figsize=(10,5))
x.plot(avg_runtime_by_year, color="yellow")
x.plot(avg_runtime_min, color="green")
x.plot(avg_runtime_max, color="green")
x.fill_between(grouped_music.index, avg_runtime_min, avg_runtime_max, color="green")
x.set_title('The average of music duration over the years', fontsize=20)
x.set_xlabel('release year', fontsize=15)
x.set_ylabel('minutes', fontsize=15)
plt.show()

결과해석:
지난 100년간 곡의 길이는 평균 3,4분 안팎을 벗어나지 않는다. 1920부터 1960년까지는 곡 길이가 꽤 들쭉날쭉하다 1980년대에 들어서 비교적 일정해지는 것을 확인할 수 있다. 

In [ ]:
plt.figure(figsize=(16,4))
sns.set(style="darkgrid")
x=df.groupby("name")["duration_min"].max().sort_values(ascending=False).head(20)
ax=sns.barplot(x.index, x)
ax.set_title('Top 20 songs with highest duration', fontsize=20)
ax.set_ylabel('duration (by minutes)', fontsize=15)
ax.set_xlabel('songs', fontsize=15)
plt.xticks(rotation=90)

결과해석:
가장 곡의 길이가 길었던 곡은 Brown Noise의 '90 Minutes'라는 노래다. 2위도 Brown Noise의 곡으로 대략 70분을 차지한다.(3위와 노래 길이는 동일)

In [ ]:
xs=df[['name','artists','release_date','popularity']].sort_values(by='popularity',ascending=False)
kpops = xs['name'].str.contains("90 Minutes|Brown Noise for Sleep|Midnight Thunderstorm Part 2|Ocean Waves for Baby Sleep")
kpop = xs[kpops]
kpop.head(10)

결과 해석: 곡의 길이가 가장 긴 상위 5개 노래를 따로 조회해본 결과, 곡 길이와 인기도와의 연관성은 떨어져보인다. 즉, 곡이 길다고해서 인기가 높은 건 아닐 것으로 추측된다. 

In [ ]:
#인기도와 음악요소 상관관계 히트맵

plt.figure(figsize=(16,8))
sns.set(style="whitegrid")
corr=df.corr()
sns.heatmap(corr, annot=True)

히트맵 결과 해석
- 인기도(popularity)와 양의 상관관계가 가장 높은 음악요소는 'energy(0.49,활기참)'와 'loudness(0.46, 높은 음량)' 순. 'danceability'(0.2, 댄스에 적합)'이 그 뒤를 잇는다. 활기찬 음악적 요소가 들어가야 노래의 인기가 높다는 걸 알 수 있다. 
- 반면 인기도와 음의 상관관계에 있는 요소는 acousticness(-0.57, 어쿠스틱), speechiness(-0.17, 보컬). 즉 비교적 조용하고 차분한 음악과 보컬 위주의 노래는 인기도와 거리가 먼 것으로 확인.
(*year은 연관해석 어려움으로 제외)

In [ ]:
plt.figure(figsize=(16,8))
sns.set(style="darkgrid")
columns=["energy","acousticness"]
for col in columns:
    x=df.groupby("year")[col].mean()
    ax=sns.lineplot(x=x.index, y=x,label=col)
ax.set_title('The drift of energetic and acoustic songs over the years', fontsize=20)
ax.set_ylabel('count', fontsize=15)
ax.set_xlabel('year', fontsize=15)

결과해석:
위 차트에서도 알 수 있듯 1900년대 초중반에는 조용하고 차분한 어쿠스틱 음악이 인기를 끌었다면, 1960년대를 기점으로 활기찬 음악이 주를 이루는 것을 확인할 수 있다. 음악 소비자들의 성향이 비트가 빠르고 에너지를 주는 음악을 더 선호한다는 걸 짐작할 수 있다. 


In [ ]:
plt.figure(figsize=(16,8))
sns.set(style="darkgrid")
columns=["tempo"]
for col in columns:
    x=df.groupby("year")[col].mean()
    ax=sns.lineplot(x=x.index, y=x, label=col)
ax.set_title('tempo', fontsize=20)
ax.set_ylabel('count', fontsize=15)
ax.set_xlabel('year', fontsize=15)


해석: 1960년대부터 활기찬 음악이 주를 이루게 되면서 음악의 템포 또한 급격히 올라가는 걸 알 수 있다. 1960년부터 본격적으로 음악의 템포가 100 이상을 상회하면서 계속 증가하고 있는 추세. 

In [ ]:
xs=df[['name','artists','release_date','popularity','tempo']].sort_values(by='popularity',ascending=False)
kpops = xs['artists'].str.contains("BLACKPINK|BTS|TWICE|ITZY")
kpop = xs[kpops]
kpop.head(25)

[번외]
세계 음악시장에서 한 획을 긋고 있는 인기 아이돌그룹 블랙핑크,BTS,트와이스,ITZY(있지) 등의 인기 음악도 모두 템포가 100 이상을 넘는다. (유일하게 템포가 100 이하인 BTS-life goes on의 경우, 그룹의 인지도가 정점에 달한 2020년 후반에 나온 곡이라 노래보다는 그룹 인기에 의해서 곡의 인기도가 높은 것으로 추정)


In [ ]:
xs=df[['name','artists','release_date','popularity','tempo']].sort_values(by='popularity',ascending=False)
kpops = xs['artists'].str.contains("Bieber|Ariana")
kpop = xs[kpops]
kpop.head(25)

빌보드 음원 강자인 해외가수 저스틴 비버와 아리아나 그란데의 인기곡들도 대부분 템포 100 이상을 유지함. 

결론: 
스포티파이에 등록된 약 17만개의 노래 데이터로 지난 100년간 음악 추이를 훑어봤다. 소비자들의 음악 취향은 시대적으로 달랐지만, 3~4분 안팎의 노래를 공통적으로 선호하는 것으로 보였다. 
또 1960년을 기점으로 템포가 빠른 곡들이 점차 인기를 끈 것으로 확인된다. kpop 가수들도 이 추이에 발맞춰 화려한 퍼포먼스를 뽐낼 수 있는 빠른 템포로 세계음원시장에 진입하고 있었다. 이런 추이로 보아 당분간 빠른 템포의 음악이 세계 음원 차트를 장악할 것으로 예측된다.
이 분석을 하면서 아쉬운 점도 있다. 인기도가 비교적 최근 발매된 곡에 집중돼 있어 1900년대 초중반 노래의 인기도는 정확히 확인할 수 없었다. 이는 스포티파이의 서비스 출시년도(2008년) 때문으로 짐작되는데 이 때문에 데이터가 고르게 분포돼 있지 못해 아쉬운 점으로 남는다.  





